# Gauss Seidel aplicado a la Ingeniería de Reactores

## Problema de Balance en Reactores consecutivos

Una reacción qúimica tiene lugar en cuatro reactores CSTR en serie cuya reacción química se muestra a continuación:

 ### A ===ki====> B

Las condiciones operativas de cada reacator se muestran en el archivo llamado "Condiciones.csv" que se muestra debajo:

In [1]:
! type Condiciones.csv

Reactor;Volumen, Vi (L);Rate ki (h^-1)
1;1000;0,1
2;1500;0,2
3;100;0,4
4;500;0,3


Algunas consideraciones importantes son:

1. El sistema está en estado estacionario
2. Las reacciones están en estado líquido
3. No hay cambio en el volumen o densidad de lo líquidos presentes
4. La tasa de consumo del componente A en cada reactor viene dada por la siguiente ecuación:

$(-r_{A}) = kC_{A}$

El arreglo de los rreactores se muestra en la siguiente figura:

<img src="arregloCSTR.png">

Se pide: 
- Usar el método de Gauss-Seidel para determinar la concentración de salida de cada reactor: 
CA1, CA2, CA3 y CA4

## Solución

Tomando en cuenta las consideraciones del proceso y las condiciones de opetación se tienen las siguientes ecuaciones de balance de materia:

Reactor 1:
$q_{A0}C_{A0} = q_{A0}C_{A1} + k_{1}C_{A1}V_{1}$

Reactor 2:
$q_{A0}C_{A1} + q_{A3}C_{A3} = (q_{A0} + q_{A3})C_{A2} + K_{2}C_{A2}V_{2}$

Reactor 3:
$(q_{A0} + q_{A3})C_{A2} + q_{A4}C_{A4} = (q_{A0} + q_{A3} + q_{A4})C_{A3} + k_{3}V_{3}C_{A3}$

Reactor 4:
$q_{A0} + q_{A4})C_{A3} = (q_{A0} + q_{A4})C_{A4} + k_{4}V_{4}C_{A4}$

Los valores de q correspondientes serán:

$q_{A0} = 1000 L/h $


$q_{A3} = 100 L/h $ 

$q_{A4} = 100 L/h $

$C_{A0} = 1 mol/L $ 

Sustituyendo en los balances nos queda:

$1100C_{A1} =      1000$

$1000C_{A1} + 100C_{A3} = 1100C_{A2} + (0.2)C_{A2}(1500)$

$1100C_{A2} + 100C_{A4} = 1200C_{A3} + (0.4)C_{A3}(100) $

$1100C_{A3} = 1100C_{A4} + (0.3)C_{A3}(500)$

De lo anterior se desprende un sistema de ecuaciones lineales y por tanto usaremos el método de Gauss-Seidel para su resolución

In [2]:
import numpy as np

iteration_limit = 10000


In [3]:
#Matrix initialization

#Coefficients imputs
A = np.array ([[1100,0,0,0],[1000,-1400,100,0],
             [0,1100,-1240,100],[0,0,1100,-1250]])

In [4]:
#Zeros vector
b = np.array([1000,0,0,0])

In [6]:
print ("Solucion del sistema de Ecuaciones:")
for i in range (A.shape[0]):
    row = ["{0:3g}*CA{1}".format(A[i,j], j+1) for j in 
          range(A.shape[1])]
    print("[{0}] = [{1:3g}]".format(" + ".join(row),b[i]))



Solucion del sistema de Ecuaciones:
[1100*CA1 +   0*CA2 +   0*CA3 +   0*CA4] = [1000]
[1000*CA1 + -1400*CA2 + 100*CA3 +   0*CA4] = [  0]
[  0*CA1 + 1100*CA2 + -1240*CA3 + 100*CA4] = [  0]
[  0*CA1 +   0*CA2 + 1100*CA3 + -1250*CA4] = [  0]


In [7]:
CA = np.array([0.1,0.1,0.1,0.1]) # you must add initial values
for it_count in range (1,iteration_limit):
    x_new =np.zeros_like(CA)
    print("Iteración {0}:{1}".format(it_count,CA))
    for i in range(A.shape[0]):
        s1= np.dot(A[i,:i], x_new[:i])
        s2 = np.dot(A[i, i + 1:], CA[i+1:])
        x_new[i] = (b[i] - s1 - s2) / A[i,i]
    if np.allclose(CA,x_new,rtol=1e-15):
        break
    CA = x_new
    
    

Iteración 1:[0.1 0.1 0.1 0.1]
Iteración 2:[0.90909091 0.65649351 0.59043779 0.51958525]
Iteración 3:[0.90909091 0.69152478 0.65535144 0.57670926]
Iteración 4:[0.90909091 0.69616147 0.6640714  0.58438283]
Iteración 5:[0.90909091 0.69678432 0.66524277 0.58541364]
Iteración 6:[0.90909091 0.69686799 0.66540012 0.58555211]
Iteración 7:[0.90909091 0.69687923 0.66542126 0.58557071]
Iteración 8:[0.90909091 0.69688074 0.6654241  0.58557321]
Iteración 9:[0.90909091 0.69688094 0.66542448 0.58557354]
Iteración 10:[0.90909091 0.69688097 0.66542453 0.58557359]


In [8]:
print("La solución es:{0}".format(CA))
error = np.dot(A,CA) - b
print("Error: {0}".format(error))

La solución es:[0.90909091 0.69688097 0.66542453 0.58557359]
Error: [0.00000000e+00 5.12374828e-06 4.50889852e-06 1.13686838e-13]
